In [1]:
import joblib

In [2]:
import pickle

In [3]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
#from sklearn.externals import joblib
import seaborn as sns
sns.set(color_codes=True)
import matplotlib.pyplot as plt
%matplotlib inline

from numpy.random import seed
from tensorflow import set_random_seed
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

from keras.layers import Input, Dropout, Dense, LSTM, TimeDistributed, RepeatVector
from keras.models import Model
from keras import regularizers

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
seed(10)
set_random_seed(10)

In [ ]:
%%time
with open('754246db-d587-4f74-8bd6-a2bc7004de76.pkl', 'rb') as f:
    data = pickle.load(f)

In [ ]:
def dataset_freq(data, freq='1min'):
    
    poc_data = data.copy()
    
    poc_data.rename(columns={'micro_data_ac_dt':'timestamp'},inplace=True)
    poc_data.dropna(inplace=True)
    poc_data['timestamp'] = pd.to_datetime(poc_data['timestamp'].astype('int'), format='%Y%m%d%H%M%S' )
    #poc_data['timestamp'] = pd.to_datetime(poc_data['timestamp'].astype('int'), format='%Y-%m-%d %H:%M:%S' )

    poc_data = poc_data.set_index('timestamp').sort_index()
    poc_data = poc_data[poc_data.tag_223m1_cont != 0]
    return poc_data.resample(freq).mean().dropna()

# shingle data
def shingle(data, shingle_size):
    num_data = len(data)
    shingled_data = np.zeros((num_data-shingle_size, shingle_size))
    
    for n in range(num_data - shingle_size):
        shingled_data[n] = data[n:(n+shingle_size)]
    return shingled_data

In [ ]:
train_data = dataset_freq(data, freq='1min')

In [ ]:
test = train_data['2020-4-28']

In [ ]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(train_data)
X_test = scaler.transform(test)
scaler_filename = "scaler_data"
joblib.dump(scaler, scaler_filename)

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
print("Training data shape:", X_train.shape)
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])
print("Test data shape:", X_test.shape)

In [ ]:
def autoencoder_model(X):
    inputs = Input(shape=(X.shape[1], X.shape[2]))
    L1 = LSTM(16, activation='relu', return_sequences=True, 
              kernel_regularizer=regularizers.l2(0.00))(inputs)
    L2 = LSTM(4, activation='relu', return_sequences=False)(L1)
    L3 = RepeatVector(X.shape[1])(L2)
    L4 = LSTM(4, activation='relu', return_sequences=True)(L3)
    L5 = LSTM(16, activation='relu', return_sequences=True)(L4)
    output = TimeDistributed(Dense(X.shape[2]))(L5)    
    model = Model(inputs=inputs, outputs=output)
    return model

In [ ]:
model = autoencoder_model(X_train)
model.compile(optimizer='adam', loss='mae')
model.summary()

In [ ]:
# fit the model to the data
nb_epochs = 1
batch_size = 10
history = model.fit(X_train, X_train, epochs=nb_epochs, batch_size=batch_size,
                    validation_split=0.05).history


In [ ]:
# plot the loss distribution of the training set
X_pred = model.predict(X_train)
X_pred = X_pred.reshape(X_pred.shape[0], X_pred.shape[2])
X_pred = pd.DataFrame(X_pred, columns=train.columns)
X_pred.index = train.index

scored = pd.DataFrame(index=train.index)
Xtrain = X_train.reshape(X_train.shape[0], X_train.shape[2])
scored['Loss_mae'] = np.mean(np.abs(X_pred-Xtrain), axis = 1)
plt.figure(figsize=(16,9), dpi=80)
plt.title('Loss Distribution', fontsize=16)
sns.distplot(scored['Loss_mae'], bins = 20, kde= True, color = 'blue');
plt.xlim([0.0,.5])

In [ ]:
#reload trained model
from tensorflow import keras
model = keras.models.load_model("Cloud_model.h5")

In [ ]:
X_pred = model.predict(X_test)
X_pred = X_pred.reshape(X_pred.shape[0], X_pred.shape[2])
X_pred = pd.DataFrame(X_pred, columns=test.columns)
X_pred.index = test.index

scored = pd.DataFrame(index=test.index)
Xtest = X_test.reshape(X_test.shape[0], X_test.shape[2])
scored['Loss_mae'] = np.mean(np.abs(X_pred-Xtest), axis = 1)
scored['Threshold'] = 0.1
scored['Anomaly'] = scored['Loss_mae'] > scored['Threshold']
scored.head()

In [ ]:
scored.plot(logy=True,  figsize=(16,9), ylim=[1e-2,1e0], color=['blue','red'])

In [ ]:
# save all model information, including weights, in h5 format
model.save("Cloud_model.h5")
print("Model saved")
